In [27]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [28]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

In [29]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


In [30]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt


In [31]:
def messages_for(website):
    system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]


In [32]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )

    return response.choices[0].message.content

In [33]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))


In [37]:
# url = "https://www.prosigmaka.com"
# url = "https://www.prodemy.id"
url = "http://www.detik.com"

display_summary(url)

# Summary of Detikcom

**Detikcom** is a leading Indonesian news website that provides the latest reliable news and updates across various categories including politics, finance, technology, health, sports, and lifestyle.

### Recent News Highlights

- **Alvin Lim's Passing**: Notable lawyer Alvin Lim has passed away, with related articles discussing his legacy and previous contributions.
- **Football Updates**: Liverpool and Manchester United have agreed to proceed with their scheduled match despite adverse weather conditions.
- **Cultural Critique**: The drama "When The Phone Rings" has faced criticism for its portrayal of Palestine.
- **Accident Reports**: A serious multi-vehicle accident on the Cipularang toll road has been reported.
- **Health Concerns**: Malaysia has recorded 327 cases of HMPV, a respiratory illness trending in China.

### Additional Noteworthy Developments

- **Consumer News**: There are significant discounts on large-sized "kecap" at Transmart.
- **Cultural Events**: The website also features event information and suggests exciting activities for audiences looking for entertainment.

The site includes various sections dedicated to different interests, making it a comprehensive source for Indonesian news and entertainment.